## Tasking Manager Monthly Report

In [4]:
import base64
import requests, json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import interact
import ipywidgets as widgets

from itsdangerous import URLSafeTimedSerializer
from IPython.display import display

/bin/sh: pip: command not found


ModuleNotFoundError: No module named 'plotly'

In [ ]:
TM_APP_BASE_URL = 'https://tasking-manager-tm4-production-api.hotosm.org'
#change for user token 
TOKEN_USER = 'Token T0RNME9EZ3ouWHpILVlBLmNhRUg0TVF0dEFWbVJLa21TRmpuWHJJMF9SYw=='

In [ ]:
headers = {
        'Authorization': TOKEN_USER,
        'Accept-Language': 'en'
        }
columns = ['project_id','project_name','project_organisation','teams','percentege_mapped',
          'percentege_validated','id_presets','instructions']
table = []

In [ ]:
def get_project_id(project_id):
    data_result={
        'project_id':[],
        'project_name':[],
        'project_organisation':[],
        'percentege_mapped':[],
        'percentege_validated':[],
        'id_presets':[],
        'instructions':[],
        'teams':[]
    }  
    result = requests.get(TM_APP_BASE_URL + '/api/v2/projects/{project_id}/'.format(project_id=project_id), headers=headers).json()
    data_result['project_id'].append(project_id)
    data_result['project_name'].append(result['projectInfo']['name'])
    data_result['project_organisation'].append(result['organisationName'])
    data_result['percentege_mapped'].append(result['percentMapped'])
    data_result['percentege_validated'].append(result['percentValidated'])
    data_result['id_presets'].append(result['idPresets'])
    data_result['instructions'].append(len(result['projectInfo']['instructions']))
    data_result['teams'].append(result['teams'])    
    return data_result

In [5]:
def btn_eventhandler(object):    
    result = get_project_id(project_id.value)
    table.append(result)

In [8]:
project_id = widgets.Text()
display(project_id)
btn = widgets.Button(description='Results')
btn.on_click(btn_eventhandler)
display(btn)


Text(value='')

Button(description='Results', style=ButtonStyle())

In [9]:
df = pd.DataFrame(table[0])
df.head()

,project_id,project_name,project_organisation,percentege_mapped,percentege_validated,id_presets,instructions,teams
0,8997,"COVID-19 - Yelimane Cercle 5 - Kayes Region, Mali",HOT,82,26,None,2748,"[{'teamId': 1, 'name': 'Validator Trainees', '..."


In [10]:
def get_all_projects():
    projects = requests.get(TM_APP_BASE_URL + '/api/v2/projects/', headers=headers).json()
    return projects

In [ ]:
table = []
data = get_all_projects()

total_projects =  data['mapResults']['features']

for value in total_projects:
    project_id = value['properties']['projectId']
    table.append(get_project_id(project_id))

In [ ]:
df = pd.DataFrame(table)
df.head()

In [ ]:
df.to_csv('list_projects.csv') 